## 处理数据

In [15]:
from collections import defaultdict
import pandas as pd
import os
import numpy as np

In [16]:
now_phase = 6
user_path=os.path.expanduser('~')   #用户目录
train_path = os.path.join(user_path, r'kdd\data\underexpose_train')
test_path = os.path.join(user_path, r'kdd\data\underexpose_test')
click=pd.DataFrame()
item_deg = defaultdict(lambda: 0) #item_deg是一个字典key=item_id, value=item_id的数量
# answer_fname文件存储每个阶段线下测试集数据，格式phase_id, user_id, item_id, item_deg[item_id]
answer_fname=r'data\debias_track_answer_%d.csv'
user_items_dict_file=r'data\user_items_dict_%d.txt'
item_feat_file=os.path.join(train_path,r'underexpose_item_feat.csv')
item_set_for_feat=set()
interacted_user_items_dict=dict()
item_set=set()

### 获取用户-商品列表的字典

In [17]:
def get_user_items_dict(underline_train, phase):
    underline_train=click.append(underline_test).drop_duplicates(keep=False)  #当前阶段以及之前阶段训练集click数据
    user_item_ = underline_train.groupby('user_id')['item_id'].agg(list).reset_index()
    # user_items_dict是一个字典，key=user_id,value=用户点击的item列表
    user_items_dict = dict(zip(user_item_['user_id'], user_item_['item_id']))
    with open(user_items_dict_file % phase, 'w') as fin:
        temp=str(user_items_dict)
        fin.write(temp)
    return user_items_dict

### 获取每个阶段top500 click item列表

In [18]:
def get_hot_list_500(underline_train, phase):
    top500_click = underline_train['item_id'].value_counts().index[:500].values  # 最热商品
    hot_list = list(top500_click)
    with open(r'data\item_topk500_%d' %phase,'w') as f:
        temp=str(hot_list)
        f.write(temp)
    return hot_list

### 获取线下测试集underline_test中用户购买的item

In [19]:
def get_underline_test_answer(underline_test):
    with open(answer_fname % c, 'w') as fout:
        for i, row in underline_test.iterrows():
            user_id, item_id, timestamp = (int(row[0]), int(row[1]), row[2])
            assert user_id % 11 == c
            print(c, user_id, item_id, item_deg[item_id], sep=',', file=fout)

### 获取item-counts字典

In [20]:
def get_item_counts(underline_train):
    temp=underline_train.groupby('item_id',as_index=False)['item_id'].agg({'item_id':'count'})
    print(temp.head())

### 数据处理主要执行块

In [21]:
for c in range(0,now_phase + 1):
    print('phase:', c)
    click_train = pd.read_csv(train_path + '\\underexpose_train_click-{}.csv'.format(c), header=None,nrows=None,
                              names=['user_id', 'item_id', 'time'],
                              dtype={'user_id':np.int, 'item_id':np.int, 'time':np.float32})
    click_test = pd.read_csv(test_path + '\\underexpose_test_click-{}.csv'.format(c), header=None,nrows=None,
                             names=['user_id', 'item_id', 'time'],
                             dtype={'user_id': np.int, 'item_id': np.int, 'time': np.float32})

    user_test=set(click_test['user_id'])  # 每阶段线下测试集用户集合
    print('len(user_val):', len(user_test))

    click_train_test = click_train.append(click_test)
    click=click.append(click_train_test)  # 当前阶段以及之前阶段click数据
    click = click.sort_values('time')       # 时间排序
    click= click.drop_duplicates(['user_id', 'item_id', 'time'], keep='last') # 去重
    item_set.update(set(click['item_id'].values))
    
    # 获取当前阶段以及之前阶段item_deg字典，item_id对应的点击次数，用作ndcg_topk_half和hitrate_topk_half统计
    temp_=click
    temp_=temp_.groupby(['item_id'], as_index=False).agg({'user_id':'count'})
    temp_.columns=['item_id','count']
    for item_id,item_count in zip(list(temp_['item_id'].values), list(temp_['count'].values)):
        item_deg[item_id]+=item_count

    click['pred'] = click['user_id'].map(lambda x: 'test' if x in user_test else 'train')
    underline_test=click[click['pred']=='test'].drop_duplicates(['user_id'], keep='last') #当前阶段线下测试集click数据
    underline_train_val = click.append(underline_test).drop_duplicates(keep=False)            #当前阶段以及之前阶段训练集click数据
    assert (underline_test['user_id'].count()+underline_train_val['user_id'].count()==click['user_id'].count())
    print('underline_test:', underline_test['user_id'].count())
#     user_items_dict=get_user_items_dict(underline_train, c)
#     hot_list=get_hot_list_500(underline_train,c)
#     get_underline_test_answer(underline_test)
    get_item_counts(underline_train_val)

#     for user_id in user_test:
#         interacted_items=user_items_dict[user_id]
#         interacted_user_items_dict[user_id]=interacted_items
# with open('data\\interacted_user_items_dict_file.txt','w') as fout:
#     temp_str=str(interacted_user_items_dict)
#     fout.write(temp_str)
print(len(item_set))

phase: 0
len(user_val): 1663
underline_test: 1663


ValueError: cannot insert item_id, already exists

In [ ]:
with open(item_feat_file,'r') as fin:
    for line in fin:
        line=line.strip().split(',[')
        item_id=int(line[0])
        item_set_for_feat.add(item_id)

In [ ]:
inner=item_set & item_set_for_feat
print(len(inner))

In [ ]:
78380 in item_set

In [ ]:
78380 in item_set_for_feat